In [1]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [21]:
import copy
import json
import os

import pandas as pd
from ipyleaflet import GeoJSON, LayersControl, Map, Popup, Marker
from ipywidgets import Layout, HTML

In [22]:
data = pd.read_csv('../analysis/data2.csv', encoding='utf8')
hpv_list = ['双价人乳头瘤病毒疫苗', '四价人乳头瘤病毒疫苗', '九价人乳头瘤病毒疫苗']

has_data_provs = data.loc[data['疫苗名称'].isin(hpv_list), '省市'].unique()
has_9_hpv_provs = data.loc[data['疫苗名称']==hpv_list[2], '省市'].unique()
has_4_hpv_provs = data.loc[data['疫苗名称']==hpv_list[1], '省市'].unique()
has_2_hpv_provs = data.loc[data['疫苗名称']==hpv_list[0], '省市'].unique()

with open(os.path.join('../src', 'zh-mainland-provinces.geojson'), 'r', encoding='utf8') as f:
    geo_data = json.load(f)

for feature in geo_data['features']:
    if feature['properties']['name_local'] is None:
        feature['properties']['name_local'] = u"青海|青海"

In [38]:
def prov_included(prov, has_provs):
    # return True if prov is in has_provs
    for a_prov in has_provs:
        if prov in a_prov:
            return True
    return False


def prepare_geo_json(geo_data):
    geo_data_copy = copy.deepcopy(geo_data)
    for idx, feature in enumerate(geo_data_copy['features']):
        name = feature['properties']['name_local']
        if not name:
            continue
        # 黑龙江繁体在前 - hardcode
        if idx != 30:
            prov = name.split("|")[-1]
        else:
            prov = name.split("|")[0]
            
        if not prov_included(prov, has_data_provs):  
            feature['properties']['style'] = {
                'color': 'black',
                'weight': 1,
                'fillColor': 'grey',
                'fillOpacity': 0.3
                }
        elif prov_included(prov, has_9_hpv_provs): 
            feature['properties']['style'] = {
                'color': 'black',
                'weight': 1,
                'fillColor': 'red',
                'fillOpacity': 0.3
                }
        elif prov_included(prov, has_4_hpv_provs):  
            feature['properties']['style'] = {
                'color': 'black',
                'weight': 1,
                'fillColor': 'yellow',
                'fillOpacity': 0.3
                }
        elif prov_included(prov, has_2_hpv_provs): 
            feature['properties']['style'] = {
                'color': 'black',
                'weight': 1,
                'fillColor': 'green',
                'fillOpacity': 0.3
                }
        else: 
            feature['properties']['style'] = {
                'color': 'black',
                'weight': 1,
                'fillColor': 'white',
                'fillOpacity': 0.3
                }
    return geo_data_copy

In [39]:
m = Map(center=(40, 105), zoom=4, layout=Layout(width='100%', height='700px'))

message = HTML()
message.value = '<ins>图解：</ins><p>红色表示该省提供HPV九价疫苗<br/>黄色表示该省提供HPV四价疫苗<br/>绿色表示该省提供HPV二价疫苗<br/>灰色表示该省暂不提供HPV疫苗。</p>'
note = Popup(
        location=(20, 135),
        child=message,
        close_button=True,
        auto_close=False,
)
m.add_layer(note)

title = Popup(
        location=(50, 105),
        child=HTML(value='<h4><b>HPV疫苗全国各省市提供情况</b></h4>'),
        close_button=False,
        auto_close=False,
)
m.add_layer(title)
geo_json = GeoJSON(data=prepare_geo_json(geo_data),
                   name='HPV')
m.add_layer(geo_json)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …